In [ ]:
!pip install kaggle

In [ ]:
# Setup Kaggle API for downloading the dataset
from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip the dataset
!kaggle datasets download -d vishnurapps/quora-question-pairs-similarity
!unzip quora-question-pairs-similarity.zip

Saving kaggle.json to kaggle (1).json
Dataset URL: https://www.kaggle.com/datasets/vishnurapps/quora-question-pairs-similarity
License(s): unknown
quora-question-pairs-similarity.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  quora-question-pairs-similarity.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:

import pandas as pd

# Load the dataset
data_path = 'train.csv'
quora_data = pd.read_csv(data_path)
print(quora_data.head())


   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  
1  What would happen if the Indian government sto...             0  
2  How can Internet speed be increased by hacking...             0  
3  Find the remainder when [math]23^{24}[/math] i...             0  
4            Which fish would survive in salt water?             0  


In [ ]:
# Filter and rename the columns
quora_data_filtered = quora_data[['question1', 'question2', 'is_duplicate']]
quora_data_filtered.columns = ['premise', 'hypothesis', 'label']



In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(quora_data_filtered, test_size=0.2, random_state=42)


In [ ]:
!pip install torchaudio autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 713.2 kB/s eta 0:00:00
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 5.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of autogluon-multimodal to determine which version is compatible with other requirements. This could take a while.
  Using cached autogluon-1.1.0-py3-none-any.whl (9.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.7/416.7 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from autogluon.multimodal import MultiModalPredictor

predictor = MultiModalPredictor(
    problem_type="text_similarity",
    query="premise",
    response="hypothesis",
    label="label",
    match_label=1,  # Assuming '1' means the questions are similar
    eval_metric='roc_auc',
)

# Fit the model
predictor.fit(
    train_data=train_data,
    time_limit=3600  # Set a reasonable time limit, e.g., one hour
)



No path specified. Models will be saved in: "AutogluonModels/ag-20240521_201931"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Apr 28 14:29:16 UTC 2024
CPU Count:          2
Pytorch Version:    2.1.2+cu121
CUDA Version:       CUDA is not available
Memory Avail:       10.65 GB / 12.67 GB (84.0%)
Disk Space Avail:   188.79 GB / 225.83 GB (83.6%)
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have in

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

INFO: Time limit reached. Elapsed time is 1:00:00. Signaling Trainer to stop.
INFO: Time limit reached. Elapsed time is 1:00:00. Signaling Trainer to stop.
INFO: Time limit reached. Elapsed time is 1:00:00. Signaling Trainer to stop.
INFO: Time limit reached. Elapsed time is 1:00:00. Signaling Trainer to stop.
INFO: Time limit reached. Elapsed time is 1:00:00. Signaling Trainer to stop.
INFO: Time limit reached. Elapsed time is 1:00:00. Signaling Trainer to stop.


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 138: 'val_roc_auc' reached 0.85871 (best 0.85871), saving model to '/content/AutogluonModels/ag-20240521_201931/epoch=0-step=138.ckpt' as top 3
INFO: Epoch 0, global step 138: 'val_roc_auc' reached 0.85871 (best 0.85871), saving model to '/content/AutogluonModels/ag-20240521_201931/epoch=0-step=138.ckpt' as top 3
INFO: Epoch 0, global step 138: 'val_roc_auc' reached 0.85871 (best 0.85871), saving model to '/content/AutogluonModels/ag-20240521_201931/epoch=0-step=138.ckpt' as top 3
INFO: Epoch 0, global step 138: 'val_roc_auc' reached 0.85871 (best 0.85871), saving model to '/content/AutogluonModels/ag-20240521_201931/epoch=0-step=138.ckpt' as top 3
INFO: Epoch 0, global step 138: 'val_roc_auc' reached 0.85871 (best 0.85871), saving model to '/content/AutogluonModels/ag-20240521_201931/epoch=0-step=138.ckpt' as top 3
INFO: Epoch 0, global step 138: 'val_roc_auc' reached 0.85871 (best 0.85871), saving model to '/content/AutogluonModels/ag-20240521_201931/epoch=

Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/AutogluonModels/ag-20240521_201931")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [ ]:
evaluation = predictor.evaluate(test_data)
print("Evaluation score:", evaluation)



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Evaluation score: {'roc_auc': 0.8625593025912348}


In [ ]:
new_data = pd.DataFrame.from_dict({
    "premise": ["How can I improve my English language?"],
    "hypothesis": ["What are some ways to become better at English?"]
})

predictions = predictor.predict(new_data)
print('Predicted similarity:', predictions)



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Predicted similarity: 0    1
Name: label, dtype: int64
